In [145]:
import numpy as np
import matplotlib.pyplot as plt

In [146]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
input_str = '<' + input_str + '>'
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', '\'', '>']
print(f'{letters=}')
# print(f'{input_str=}')

len(letters)=32
502
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', "'", '>']


In [147]:
# generate SDR

out_len = 128 # 128
sparsity = 4 # 4
max_overlap = 1 # 1

sparse_prob = sparsity / out_len

letter_SDRs = {}

for letter in letters:
    while True:
        rnd_SDR = np.random.choice([0, 1], out_len, True, [1 - sparse_prob, sparse_prob])

        if (rnd_SDR > 0).sum() != sparsity:
            continue
        
        is_over_max = False
        for inp_SDR in letter_SDRs.values():
            if (inp_SDR & rnd_SDR).sum() > max_overlap:
                is_over_max = True
                break
        
        if is_over_max:
            continue
            
        break
    
    # print(f'{letter}')
    letter_SDRs.setdefault(letter, rnd_SDR)

for letter, inp_SDR in letter_SDRs.items():
    print(f'{letter} : {(inp_SDR > 0).sum()} {inp_SDR.tolist()}')

a : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
b : 4 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [148]:
# text to SDRs
input_SDRs = []

for inp in input_str:
    input_SDRs.append(letter_SDRs[inp])

print(f'{len(input_SDRs)=}')

# for SDR in input_SDRs:
#     print(f'{SDR.tolist()}')

len(input_SDRs)=502


In [149]:
from numpy import ndarray

# init start column
column_size = 16

start_SDR = letter_SDRs['<']

def get_start_I(start_SDR: ndarray):
    start_I = np.zeros((start_SDR.size, column_size))

    active_start_SDR_idx = np.where(start_SDR == 1)

    for as_idx in active_start_SDR_idx[0]:
        rnd_active = np.random.randint(column_size)

        start_I[as_idx][rnd_active] = 1

    return start_I


start_I = get_start_I(start_SDR=start_SDR)

# for col in np.rot90(start_I):
#     print(f'{col.astype(int).tolist()}')


In [150]:
def train(input_SDRs: list[ndarray],
          push_delta=0.04,
          epoch_cnt=1,
          inp_len=128,
          out_len=128,
          sparsity=4,
          eps0=0.01,
          eps0_V=0.09,
          ):
    I = np.zeros(inp_len)
    K = np.zeros(out_len)

    S = np.random.rand(out_len, out_len) * 1e-2

    V = np.zeros(out_len)

    prec = 1e-30

    inputs_idx = np.arange(len(input_SDRs))

    for epoch_idx in range(epoch_cnt):
        np.random.shuffle(inputs_idx)

        # eps = (epoch_cnt - epoch_idx) * eps0

        for SDR_idx in inputs_idx:
            I = np.copy(input_SDRs[SDR_idx])

            K = np.dot(S, I)

            K /= np.max(np.abs(K)) + prec

            nV = np.abs(V - np.max(V))
            nV /= np.max(nV) + prec

            OV = K + nV * eps0_V

            max_O_idx = np.argsort(OV)[::-1][:sparsity]
            pull_idx = max_O_idx[0]
            push_idx = max_O_idx[1:sparsity]

            # eps
            eps = np.zeros(K.shape)
            eps[max_O_idx] = eps0 * ((np.linalg.norm(S[max_O_idx], axis=1) - 1)**2)

            S[pull_idx] += (I - S[pull_idx] * K[pull_idx][np.newaxis].T) * eps[pull_idx][np.newaxis].T
            S[push_idx] += (I - S[push_idx] * K[push_idx][np.newaxis].T) * (-push_delta) * eps[push_idx][np.newaxis].T

            # winners
            V[pull_idx] += 1

    print(np.sum(V == 0))

    return S, V


S, UsV = train(input_SDRs=input_SDRs,
          epoch_cnt=100,
          push_delta=0.04,
          out_len=128,
          sparsity=4,
          eps0=0.0001,
          eps0_V=0.09,)
print(f'{S.max()=} {S.min()=} {np.mean(S)=}')


99
S.max()=0.32702050332168886 S.min()=-0.016119793252005025 np.mean(S)=0.005427548567489293


In [151]:
def infer_one(S: ndarray, inp_SDR: ndarray, sparsity=4) -> tuple[ndarray, ndarray]:
    K = np.dot(S, inp_SDR)

    max_O_idx = np.argsort(K)[::-1][:sparsity]

    BK = np.zeros(K.shape)
    BK[max_O_idx] = 1

    return K, BK

for inp_SDR in input_SDRs:
    O, BK = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)

    print(BK.astype(int).tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [152]:
def predict_train_one(W: ndarray,
                      V: ndarray,
                      S: ndarray,
                      inp_SDR: ndarray,
                      I: ndarray,
                      sparsity=4,
                      prec=1e-30,
                      eps0_V=0.09,
                      eps0=0.003,):
    K, SDR = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)

    I = I.reshape(out_len * column_size)

    O = np.dot(W, I)

    O /= np.max(np.abs(O)) + prec

    nV = np.abs(V - np.max(V, axis=1)[np.newaxis].T)
    nV /= np.max(nV, axis=1)[np.newaxis].T + prec

    OV = O + nV * eps0_V

    OV *= SDR[np.newaxis].T
    max_O = np.max(OV, axis=1)

    O *= SDR[np.newaxis].T
    O[OV < max_O[np.newaxis].T] = 0
    max_O_idx = np.where(O != 0)

    W[max_O_idx] += (I - W[max_O_idx] * O[max_O_idx][..., np.newaxis]) * eps0

    # winners
    V[max_O_idx] += 1

    # recur
    O[O != 0] = 1

    I = np.copy(O)

    return I


In [153]:
prec = 1e-30

V = np.zeros((out_len, column_size))

def train_predict(input_SDRs: list[ndarray],
                  S: ndarray,
                  V: ndarray,
                  start_I: ndarray,
                  out_len,
                  column_size,
                  prec=1e-30,
                  epoch_cnt=1,
                  sparsity=4,):
    I = np.zeros((out_len, column_size))
    O = np.zeros((out_len, column_size))

    W = np.random.rand(out_len, column_size, out_len * column_size) * 1e-2

    eps0 = 0.003

    eps0_V = 0.09

    for epoch_idx in range(epoch_cnt):
        I = np.copy(start_I)

        for inp_SDR in input_SDRs[1:]:
            K, SDR = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)

            I = I.reshape(out_len * column_size)

            O = np.dot(W, I)

            O /= np.max(np.abs(O)) + prec

            nV = np.abs(V - np.max(V, axis=1)[np.newaxis].T)
            nV /= np.max(nV, axis=1)[np.newaxis].T + prec

            OV = O + nV * eps0_V

            OV *= SDR[np.newaxis].T
            max_O = np.max(OV, axis=1)

            O *= SDR[np.newaxis].T
            O[OV < max_O[np.newaxis].T] = 0
            max_O_idx = np.where(O != 0)

            W[max_O_idx] += (I - W[max_O_idx] * O[max_O_idx][..., np.newaxis]) * eps0

            # winners
            V[max_O_idx] += 1

            # recur
            O[O != 0] = 1

            I = np.copy(O)

    # O.shape
    print(np.mean(V, axis=1).tolist())
    print(np.sum(V == 0))
    print(np.sum((V > 0), axis=1).tolist())
    # for col in np.rot90(V):
    #     print(f'{col.astype(int).tolist()}')

    return W, V


W, V = train_predict(input_SDRs=input_SDRs,
                     S=S,
                     V=V,
                     start_I=start_I,
                     epoch_cnt=1,
                     prec=prec,
                     out_len=out_len,
                     column_size=column_size,
                     sparsity=sparsity,)


[0.5, 6.5625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.5, 0.25, 0.0, 3.625, 0.0, 0.0, 0.0, 0.6875, 0.0, 0.0, 0.0, 0.0, 6.0625, 3.6875, 5.3125, 4.1875, 0.0, 0.0, 0.0, 0.0, 5.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.375, 0.0, 0.0, 0.0, 9.8125, 0.0, 0.0, 0.0, 0.0625, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 11.75, 2.875, 2.0625, 7.0625, 0.0, 3.4375, 0.5625, 0.0, 0.0, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.8125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.375, 0.0, 7.625, 0.0, 7.8125, 0.0, 0.0, 3.8125, 0.0, 0.0, 0.0, 0.0]
1654
[7, 16, 0, 0, 0, 0, 0, 0, 15, 4, 0, 15, 0, 0, 0, 9, 0, 0, 0, 0, 16, 16, 16, 16, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 16, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 4, 16, 15, 15, 16, 0, 16, 7, 0, 0, 1, 0, 0, 0, 0, 0,

In [154]:
rev_start_I = get_start_I(start_SDR=letter_SDRs['>'])

RV = np.zeros((out_len, column_size))

RW, RV = train_predict(input_SDRs=input_SDRs[::-1],
                       S=S,
                       V=RV,
                       start_I=rev_start_I,
                       epoch_cnt=1,
                       prec=prec,
                       out_len=out_len,
                       column_size=column_size,
                       sparsity=sparsity,)


[0.5, 6.5625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.5, 0.25, 0.0, 3.6875, 0.0, 0.0, 0.0, 0.6875, 0.0, 0.0, 0.0, 0.0, 6.0625, 3.6875, 5.3125, 4.1875, 0.0, 0.0, 0.0, 0.0, 5.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.8125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0625, 4.3125, 0.0, 0.0, 0.0, 9.8125, 0.0, 0.0, 0.0, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 11.75, 2.875, 2.0625, 7.0625, 0.0, 3.4375, 0.5625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.8125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.375, 0.0, 7.625, 0.0, 7.8125, 0.0, 0.0, 3.875, 0.0, 0.0, 0.0, 0.0]
1655
[8, 16, 0, 0, 0, 0, 0, 0, 16, 3, 0, 16, 0, 0, 0, 8, 0, 0, 0, 0, 16, 16, 16, 16, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 1, 15, 0, 0, 0, 16, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 16, 14, 15, 16, 0, 15, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [155]:
# test

def test_predict(S: ndarray, W: ndarray, input_SDRs: list[ndarray], start_I: ndarray, sparsity=4):
    error = []
    us_Ks = []
    prd_Ks = []

    I = np.copy(start_I)

    for inp_SDR in input_SDRs[1:]:
        K, SDR = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)
        us_Ks.append(K)

        I = I.reshape(out_len * column_size)

        O = np.dot(W, I)

        max_O = np.max(O, axis=1)
        argmax_O_idx = np.argsort(max_O)[::-1][:sparsity]

        SDR_O = np.zeros(O.shape[0])
        SDR_O[argmax_O_idx] = 1

        prd_Ks.append(max_O)

        err = (SDR.astype(int) ^ SDR_O.astype(int)).sum()
        # print(f'{err=}')
        error.append(err)

        O *= SDR[np.newaxis].T
        max_O = np.max(O, axis=1)
        O[O < max_O[np.newaxis].T] = 0

        # recur
        O[O != 0] = 1

        I = np.copy(O)

    error = np.array(error)
    mean_percent = int(np.mean(error) / (2 * sparsity) * 100)
    print(f'{np.mean(error)=} {error.max()=} {error.min()=} {mean_percent=}')

    return error, us_Ks, prd_Ks


error, us_Ks, prd_Ks = test_predict(S=S,
                                    W=W,
                                    input_SDRs=input_SDRs,
                                    start_I=start_I,
                                    sparsity=sparsity,)

rev_error, rev_us_Ks, rev_prd_Ks = test_predict(S=S,
                                                W=RW,
                                                input_SDRs=input_SDRs[::-1],
                                                start_I=rev_start_I,
                                                sparsity=sparsity,)

print(f'{len(us_Ks)=} {len(prd_Ks)=} {len(rev_prd_Ks)=}')
# O.shape
# for col in np.rot90(O):
#     print(f'{col.astype(int).tolist()}')


np.mean(error)=1.9600798403193613 error.max()=8 error.min()=0 mean_percent=24
np.mean(error)=1.748502994011976 error.max()=8 error.min()=0 mean_percent=21
len(us_Ks)=501 len(prd_Ks)=501 len(rev_prd_Ks)=501


In [156]:
a = np.arange(8)
ra = np.copy(a[::-1])
a = a[1:]
ra = ra[1:]
print(a)
print(ra[::-1][1:])

[1 2 3 4 5 6 7]
[1 2 3 4 5 6]


In [157]:
def after_train_one(K: ndarray,
                    S: ndarray,
                    dS: ndarray,
                    V: ndarray,
                    I: ndarray,
                    push_delta=0.04,
                    eps0=0.0001,
                    sparsity=4,
                    eps0_V=0.09,):
    K /= np.max(np.abs(K)) + prec

    nV = np.abs(V - np.max(V))
    nV /= np.max(nV) + prec

    OV = K + nV * eps0_V

    max_O_idx = np.argsort(OV)[::-1][:sparsity]
    pull_idx = max_O_idx[0]
    push_idx = max_O_idx[1:sparsity]

    # eps
    eps = np.zeros(K.shape)
    eps[max_O_idx] = eps0 * ((np.linalg.norm(S[max_O_idx], axis=1) - 1)**2)

    S[pull_idx] += (I - S[pull_idx] * K[pull_idx][np.newaxis].T) * eps[pull_idx][np.newaxis].T
    S[push_idx] += (I - S[push_idx] * K[push_idx][np.newaxis].T) * (-push_delta) * eps[push_idx][np.newaxis].T

    # winners
    V[pull_idx] += 1

    return dS


In [158]:
# after train
dS = np.zeros(S.shape)
aS = np.copy(S)
aW = np.copy(W)
aV = np.copy(V)
aRW = np.copy(RW)
aRV = np.copy(RV)
aUsV = np.copy(UsV)

# us_Ks
# prd_Ks
rev_prd_Ks = rev_prd_Ks[::-1][1:]

preference_feedback = 0.5

input_idx = np.arange(len(rev_prd_Ks))

epoch_cnt = 10

for epoch_idx in range(epoch_cnt):
    np.random.shuffle(input_idx)

    for inp_idx in input_idx:
        I_SDR = input_SDRs[inp_idx + 1]

        us_K = us_Ks[inp_idx] / np.max(np.abs(us_Ks[inp_idx]))
        prd_K = prd_Ks[inp_idx] / np.max(np.abs(prd_Ks[inp_idx]))
        rev_prd_K = rev_prd_Ks[inp_idx] / np.max(np.abs(rev_prd_Ks[inp_idx]))

        K = us_K + prd_K * preference_feedback + rev_prd_K * preference_feedback

        dS = after_train_one(K=K,
                             S=aS,
                             dS=dS,
                             V=aUsV,
                             I=I_SDR,)

    aW, aV = train_predict(input_SDRs=input_SDRs,
                           S=aS,
                           V=np.zeros((out_len, column_size)),
                           start_I=start_I,
                           epoch_cnt=1,
                           prec=prec,
                           out_len=out_len,
                           column_size=column_size,
                           sparsity=sparsity,)

    aRW, aRV = train_predict(input_SDRs=input_SDRs[::-1],
                             S=aS,
                             V=np.zeros((out_len, column_size)),
                             start_I=rev_start_I,
                             epoch_cnt=1,
                             prec=prec,
                             out_len=out_len,
                             column_size=column_size,
                             sparsity=sparsity,)

    error, us_Ks, prd_Ks = test_predict(S=aS,
                                        W=aW,
                                        input_SDRs=input_SDRs,
                                        start_I=start_I,
                                        sparsity=sparsity,)

    rev_error, rev_us_Ks, rev_prd_Ks = test_predict(S=aS,
                                                    W=aRW,
                                                    input_SDRs=input_SDRs[::-1],
                                                    start_I=rev_start_I,
                                                    sparsity=sparsity,)
    rev_prd_Ks = rev_prd_Ks[::-1][1:]


[0.0, 6.5625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.5, 0.0, 0.0, 3.625, 0.0, 0.0, 0.0, 0.6875, 0.0, 0.5625, 0.0, 0.0, 6.0625, 3.6875, 5.3125, 4.1875, 0.0, 0.0, 0.0, 0.0, 5.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5625, 0.0, 0.0, 3.8125, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 4.375, 0.0, 0.0, 0.0, 9.8125, 0.0, 0.25, 0.0, 0.0625, 0.0625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 11.75, 2.875, 2.0625, 7.0625, 0.0, 3.4375, 0.0625, 0.0, 0.0, 0.0625, 0.5625, 0.0, 0.0, 0.0, 0.0, 0.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.8125, 0.0, 0.0, 0.8125, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.625, 0.0, 7.8125, 0.0, 0.0, 3.8125, 0.0, 0.0, 0.0, 0.0]
1648
[0, 16, 0, 0, 0, 0, 0, 0, 16, 0, 0, 15, 0, 0, 0, 8, 0, 7, 0, 0, 16, 16, 16, 16, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 16, 0, 0, 0, 4, 0, 0, 0, 16, 0, 0, 0, 16, 0, 4, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 16, 15, 16, 0, 16, 1, 0, 0, 1, 8, 0, 0, 0

In [159]:


test_predict(S=aS,
             W=aW,
             input_SDRs=input_SDRs,
             start_I=start_I,
             sparsity=sparsity,)

np.mean(error)=1.441117764471058 error.max()=8 error.min()=0 mean_percent=18


(array([2, 2, 4, 0, 0, 2, 4, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 4, 0, 0, 2, 2,
        2, 2, 6, 2, 2, 4, 4, 0, 0, 0, 0, 0, 0, 0, 4, 4, 6, 8, 6, 2, 0, 0,
        0, 4, 2, 2, 4, 4, 0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 0, 2, 0, 4, 0, 6,
        4, 0, 4, 0, 0, 2, 2, 0, 0, 0, 8, 0, 0, 0, 4, 2, 4, 0, 4, 0, 0, 0,
        4, 0, 2, 0, 4, 0, 0, 4, 2, 6, 6, 6, 8, 6, 6, 6, 0, 2, 2, 6, 0, 0,
        2, 0, 2, 0, 0, 2, 2, 4, 0, 4, 0, 0, 2, 0, 0, 0, 4, 0, 4, 0, 0, 0,
        2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0,
        4, 0, 2, 2, 0, 0, 0, 0, 4, 4, 2, 0, 2, 0, 4, 0, 0, 2, 2, 2, 4, 2,
        0, 0, 4, 2, 0, 2, 0, 0, 0, 2, 2, 0, 2, 0, 4, 2, 2, 2, 2, 0, 4, 0,
        0, 2, 0, 0, 0, 4, 2, 2, 4, 0, 0, 2, 4, 2, 0, 0, 0, 4, 0, 2, 0, 0,
        0, 4, 0, 0, 0, 4, 2, 0, 0, 0, 4, 0, 0, 2, 0, 4, 0, 0, 0, 0, 0, 2,
        0, 0, 2, 2, 8, 0, 2, 0, 0, 2, 2, 2, 0, 0, 0, 0, 2, 2, 0, 2, 2, 4,
        2, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0,
        4, 0, 0, 4, 2, 2, 2, 2, 0, 0, 